<h1> Structured Data Solution </h1>

In this notebook, we will use the structured data package in Datalab to build a model to predict taxifares.

In [6]:
import os
PROJECT = 'cloud-training-demos'    # CHANGE THIS
BUCKET = 'cloud-training-demos-ml'  # CHANGE THIS
REGION = 'us-central1' # CHANGE THIS

os.environ['PROJECT'] = PROJECT # for bash
os.environ['BUCKET'] = BUCKET # for bash
os.environ['REGION'] = REGION # for bash

In [ ]:
%bash
echo "project=$PROJECT"
echo "bucket=$BUCKET"
echo "region=$REGION"
gcloud config set project $PROJECT
gcloud config set compute/region $REGION
gcloud beta ml init-project -q

In [1]:
import tensorflow as tf
import datalab.ml as ml
import google.cloud.ml as cml
import datalab_solutions.structured_data as sd
from tensorflow.python.lib.io import file_io
import json
import shutil

print('tf ' + str(tf.__version__))
print('sd ' + str(sd.__version__))
print('cml ' + str(cml.__version__))

No handlers could be found for logger "oauth2client.contrib.multistore_file"


tf 1.0.0
sd 0.0.1
cml 0.1.9.1-alpha


In [13]:
INDIR = '../feateng/sample'
OUTDIR = '.'

# for bash
os.environ['OUTDIR'] = OUTDIR

<h2> Set up schema file </h2>

Schema of training/test. Same format as BigQuery.  STRING/INTEGER/FLOAT only.

In [3]:
%writefile $OUTDIR/taxifare.json
[
    {
        "mode": "NULLABLE",
        "name": "fare_amount",
        "type": "FLOAT"
    }, 
    {
        "mode": "NULLABLE",
        "name": "dayofweek",
        "type": "STRING"
    },
    {
        "mode": "NULLABLE",
        "name": "hourofday",
        "type": "STRING"
    },
    {
        "mode": "NULLABLE",
        "name": "pickuplon",
        "type": "FLOAT"
    },
    {
        "mode": "NULLABLE",
        "name": "pickuplat",
        "type": "FLOAT"
    },
    {
        "mode": "NULLABLE",
        "name": "dropofflon",
        "type": "FLOAT"
    },
    {
        "mode": "NULLABLE",
        "name": "dropofflat",
        "type": "FLOAT"
    },
    {
        "mode": "NULLABLE",
        "name": "passengers",
        "type": "FLOAT"
    },
    {
        "mode": "REQUIRED",
        "name": "key",
        "type": "STRING"
    } 
]

Overwriting ./taxifare.json


<h2> Preprocessing </h2>

The first step of preprocessing is to compute the min, max, etc. for scaling purposes.

In [14]:
!rm -rf $OUTDIR/taxi_preproc $OUTDIR/taxi_trained

In [15]:
train_csv = ml.CsvDataSet(
  file_pattern=os.path.join(INDIR, 'train*'),
  schema_file=os.path.join(OUTDIR, 'taxifare.json'))
sd.local_preprocess(
  dataset=train_csv,
  output_dir=os.path.join(OUTDIR, 'taxi_preproc'),
)

Starting local preprocessing.
Local preprocessing done.


The second step is to specify the feature columns and transformations.  The target and key transforms are required. Everything else is optional.

In [16]:
transforms = {
  "fare_amount": {"transform": "target"},
  "key": {"transform": "key"}, 
  "dayofweek": {"transform": "one_hot"},
  "hourofday": {"transform": "embedding", "embedding_dim": 2}, # group-combine the hour
}
file_io.write_string_to_file(os.path.join(OUTDIR, 'taxi_preproc/transforms.json'),
                             json.dumps(transforms, indent=2))

In [17]:
%bash
ls $OUTDIR/taxi_preproc

numerical_analysis.json
schema.json
transforms.json
vocab_dayofweek.csv
vocab_hourofday.csv
vocab_key.csv


In [19]:
!cat $OUTDIR/taxi_preproc/num*json

{
  "passengers": {
    "max": 6.0,
    "mean": 1.714834159189322,
    "min": 1.0
  },
  "fare_amount": {
    "max": 130.12,
    "mean": 11.346521137449155,
    "min": 2.5
  },
  "pickuplat": {
    "max": 41.591743,
    "mean": 40.751719439603896,
    "min": 40.27725
  },
  "dropofflat": {
    "max": 41.57285,
    "mean": 40.75180645182645,
    "min": 40.303627
  },
  "pickuplon": {
    "max": -73.137393,
    "mean": -73.97476843613309,
    "min": -75.26911
  },
  "dropofflon": {
    "max": -73.137393,
    "mean": -73.97400344112786,
    "min": -74.417107
  }
}

In [20]:
!cat $OUTDIR/taxi_preproc/vocab_day*

Wed
Sun
Fri
Tue
Mon
Thu
Sat

<h2> Local Training and prediction </h2>

Train using the preproprocessed data.

In [22]:
eval_csv = ml.CsvDataSet(
  file_pattern=os.path.join(INDIR, 'valid*'),
  schema_file=os.path.join(OUTDIR, 'taxifare.json'))

shutil.rmtree(os.path.join(OUTDIR, 'taxi_trained'), ignore_errors=True)
sd.local_train(
  train_dataset=train_csv,
  eval_dataset=eval_csv,
  preprocess_output_dir=os.path.join(OUTDIR, 'taxi_preproc'),
  transforms=os.path.join(OUTDIR, 'taxi_preproc/transforms.json'),
  output_dir=os.path.join(OUTDIR, 'taxi_trained'),
  model_type='dnn_regression',
  max_steps=2500,
  layer_sizes=[64, 4]
)

Starting local training.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5e98bf3d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5e98bf3d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


...

INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: ./taxi_trained/train/export/intermediate_prediction_models/1488299893441/saved_model.pb


INFO:tensorflow:SavedModel written to: ./taxi_trained/train/export/intermediate_prediction_models/1488299893441/saved_model.pb


Local training done.


In [23]:
!ls taxi_trained

evaluation_model  model  train


In [25]:
sd.local_predict(
  training_ouput_dir=os.path.join(OUTDIR, 'taxi_trained'),
  data=['Sun,0,-73.984685,40.769262,-73.991065,40.728145,5.0,row_01',
        'Mon,12,-74.006927,40.739993,-73.950025,40.773403,1.0,row_02',
        'Tues,8,-73.977345,40.779387,-73.97615,40.778867,1.0,row_03',
        'Fri,17,-73.97136,40.794413,-73.99623,40.74524,1.0,row_04',
        'Sun,0,-73.997642,40.763853,-73.99485,40.750282,1.0,row_05',
        'Sun,0,-74.004538,40.742202,-73.955823,40.773485,1.0,row_06',
        'Sun,0,-74.000589,40.73731,-73.985902,40.692725,1.0,row_07',
        'Sun,0,-73.995432,40.72114,-73.992403,40.719745,1.0,row_08',
        'Sun,0,-73.945033,40.779203,-73.952037,40.766802,1.0,row_09',
        'Sun,0,-73.968592,40.693262,-73.99231,40.694317,1.0,row_10']
)

Starting local prediction.
Local prediction done.


,key_from_input,predicted_target
0,row_01,11.724672
1,row_02,11.658803
2,row_03,11.299863
3,row_04,11.723350
4,row_05,11.683121
5,row_06,11.679772
6,row_07,11.680917
7,row_08,11.680292
8,row_09,11.678738
9,row_10,11.677299


In [26]:
shutil.rmtree(os.path.join(OUTDIR,'batch_predict'), ignore_errors=True)
sd.local_batch_predict(
  training_ouput_dir=os.path.join(OUTDIR, 'taxi_trained'),
  prediction_input_file=os.path.join(INDIR, 'valid*'),
  output_dir=os.path.join(OUTDIR, 'batch_predict'),
  output_format='csv',
  mode='prediction'
)

Starting local batch prediction.
Local batch prediction done.


In [31]:
!head $OUTDIR/batch_predict/errors-00000-of-00022.txt

('Expect 8 fields but have 9 in record 0\n\t [[Node: csv_to_tensors = DecodeCSV[OUT_TYPE=[DT_STRING, DT_STRING, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_STRING], field_delim=",", _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_csv_input_string_0, csv_to_tensors/record_defaults_0, csv_to_tensors/record_defaults_1, csv_to_tensors/record_defaults_2, csv_to_tensors/record_defaults_3, csv_to_tensors/record_defaults_4, csv_to_tensors/record_defaults_5, csv_to_tensors/record_defaults_6, csv_to_tensors/record_defaults_7)]]\n\nCaused by op u\'csv_to_tensors\', defined at:\n  File "/usr/lib/python2.7/threading.py", line 783, in __bootstrap\n    self.__bootstrap_inner()\n  File "/usr/lib/python2.7/threading.py", line 810, in __bootstrap_inner\n    self.run()\n  File "/usr/local/lib/python2.7/dist-packages/apache_beam/runners/direct/executor.py", line 84, in run\n    task()\n  File "/usr/local/lib/python2.7/dist-packages/apache_beam/runners/direct/executor.py", line 292, in __ca

<h2> Cloud preprocessing and training </h2>

In the above cells, change INDIR and OUTDIR to be GCS.

Change the calls from local_predict to cloud_predict. That's it.



Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License